<h1 style="color:#520080">Análisis de Homicidios</h1>

<h4 style="color:#946CFF">Importamos librerías</h4>

In [2]:
import pandas as pd
df = pd.read_csv("database.csv.zip")
#Hemos escogido un CSV muy completo, partiremos con el para hacer las tablas que necesitamos.

/var/folders/4q/413qpbj17ks9bgdzl52cpx9m0000gn/T/ipykernel_8946/2651182518.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("database.csv.zip")


<h3 style="color:#946CFF">Inspección</h3>

In [3]:
df.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


In [4]:
df.shape

(638454, 24)

In [5]:
df.columns

Index(['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime Type', 'Crime Solved',
       'Victim Sex', 'Victim Age', 'Victim Race', 'Victim Ethnicity',
       'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race',
       'Perpetrator Ethnicity', 'Relationship', 'Weapon', 'Victim Count',
       'Perpetrator Count', 'Record Source'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638454 entries, 0 to 638453
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   Record ID              638454 non-null  int64 
 1   Agency Code            638454 non-null  object
 2   Agency Name            638454 non-null  object
 3   Agency Type            638454 non-null  object
 4   City                   638454 non-null  object
 5   State                  638454 non-null  object
 6   Year                   638454 non-null  int64 
 7   Month                  638454 non-null  object
 8   Incident               638454 non-null  int64 
 9   Crime Type             638454 non-null  object
 10  Crime Solved           638454 non-null  object
 11  Victim Sex             638454 non-null  object
 12  Victim Age             638454 non-null  int64 
 13  Victim Race            638454 non-null  object
 14  Victim Ethnicity       638454 non-null  object
 15  

In [7]:
df.dtypes

Record ID                 int64
Agency Code              object
Agency Name              object
Agency Type              object
City                     object
State                    object
Year                      int64
Month                    object
Incident                  int64
Crime Type               object
Crime Solved             object
Victim Sex               object
Victim Age                int64
Victim Race              object
Victim Ethnicity         object
Perpetrator Sex          object
Perpetrator Age          object
Perpetrator Race         object
Perpetrator Ethnicity    object
Relationship             object
Weapon                   object
Victim Count              int64
Perpetrator Count         int64
Record Source            object
dtype: object

In [8]:
df["Perpetrator Age"]=pd.to_numeric(df["Perpetrator Age"], errors='coerce')

In [9]:
df["Perpetrator Age"]=df["Perpetrator Age"].fillna(0).astype(int)

In [10]:
df.describe()

,Record ID,Year,Incident,Victim Age,Perpetrator Age,Victim Count,Perpetrator Count
count,638454.00000,638454.000000,638454.000000,638454.000000,638454.000000,638454.000000,638454.000000
mean,319227.50000,1995.801102,22.967924,35.033512,20.322665,0.123334,0.185224
std,184305.93872,9.927693,92.149821,41.628306,17.886846,0.537733,0.585496
min,1.00000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,159614.25000,1987.000000,1.000000,22.000000,0.000000,0.000000,0.000000
50%,319227.50000,1995.000000,2.000000,30.000000,21.000000,0.000000,0.000000
75%,478840.75000,2004.000000,10.000000,42.000000,31.000000,0.000000,0.000000
max,638454.00000,2014.000000,999.000000,998.000000,99.000000,10.000000,10.000000


In [13]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.strip()

In [ ]:
df.columns

Index(['record_id', 'agency_code', 'agency_name', 'agency_type', 'city',
       'state', 'year', 'month', 'incident', 'crime_type', 'crime_solved',
       'victim_sex', 'victim_age', 'victim_race', 'victim_ethnicity',
       'perpetrator_sex', 'perpetrator_age', 'perpetrator_race',
       'perpetrator_ethnicity', 'relationship', 'weapon', 'victim_count',
       'perpetrator_count', 'record_source'],
      dtype='object')

In [19]:
df.duplicated().sum()

0

In [ ]:
df.isnull().sum()

<h3 style="color:#946CFF">División de tablas</h3>

In [16]:
df['victim_id'] = 'VIC' + df.index.astype(str)
df['perpetrator_id'] = 'PER' + df.index.astype(str)
df['relationship_id'] = 'REL' + df.index.astype(str)

In [24]:
agencies = df[['agency_code', 'agency_name', 'agency_type', 'city', 'state']]
incident = df[['record_id', 'agency_code', 'victim_id', 'perpetrator_id', 'relationship_id', 'year', 'month', 'crime_type', 'crime_solved', 'weapon']]
victims = df[['victim_id', 'record_id', 'victim_sex', 'victim_age', 'victim_race', 'victim_ethnicity', 'victim_count']]
perpetrators = df[['perpetrator_id', 'record_id', 'perpetrator_sex', 'perpetrator_age', 'perpetrator_race', 'perpetrator_ethnicity', 'perpetrator_count']]
relationship = df[['relationship_id', 'record_id', 'relationship']]


DFS GUARDADOS COMO CSVS : 

agencies.to_csv("agencies.csv", index=False)


incident.to_csv("incident.csv", index=False)


victims.to_csv("victims.csv", index=False)


perpetrators.to_csv("perpetrators.csv", index=False)


relationship.to_csv("relationship.csv", index=False)

<h3 style="color:#946CFF">Hypotesis</h3>

1. ¿Qué características y rasgos presentan la mayoría de los delincuentes? ¿Cuál es el contexto? Debemos leer los datos con un prisma histórico y social. 
2. ¿A qué grupo sociales más vulnerables pertenecían las víctimas? ¿Cuál es su historia? 
3. ¿Qué arma es la más utilizada en estos homicidios?
4. ¿Qué relación mantenían víctima y agresor?
5. ¿Qué lugares son los más inseguros, con mayor probabilidad de homicidio?